## Running and Observing Agents

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import os

In [2]:
key = "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"

In [3]:
open_router_model_client =  OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3-0324:free",
    api_key = key,
    model_info={
        "family":'deepseek',
        "vision" :True,
        "function_calling":True,
        "json_output": False
    }
)

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [4]:
async def web_search(query: str) -> str:
    """find info on the web"""
    return "dog has four legs two eyes one mouth one tail and two ears"

In [7]:
agent = AssistantAgent(
    name = 'assistant',
    model_client = open_router_model_client,
    tools = [web_search],
    system_message = "use tool to solve the task"
)

In [8]:
result = await agent.run(task = "find info about dog")
print(result.messages[-1].content)

dog has four legs two eyes one mouth one tail and two ears


how to actually see that we have used tool above
### on_messsage() method

In [15]:
from autogen_core import CancellationToken #A CancellationToken lets you politely tell a running AutoGen task “please stop now,” and the task checks the token at safe points to abort cleanly. It’s like a cooperative “stop” flag for long‑running async operations.

async def assistant_run():
    response = await agent.on_messages(
        messages = [TextMessage(content = "find information about dog", source = 'User')],
        cancellation_token = CancellationToken()
    )

    print(response.inner_message) # produced by agent can be agent event
    print("\n\n\n")
    print(response.chat_message) # produced by agent as response

await assistant_run()

TypeError: 'NoneType' object is not subscriptable

### streammint messages
#### on_messages_stream()

In [20]:
import asyncio
from autogen_core import CancellationToken

from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    # 1. Build a valid user message (source must be exactly "user")
    user_msg = TextMessage(
        content="Find information about Labrador Retriever via the tool",
        source="user"     # ← lowercase
    )

    # 2. Kick off the streaming call
    stream = agent.on_messages_stream(
        messages=[user_msg],
        cancellation_token=CancellationToken()
    )

    # 3. Pass the stream into Console for auto‑printing
    await Console(
        stream,            # the async iterator
        output_stats=True  # show per‑chunk timing/stats
    )

# 4. Run it
asyncio.run(assistant_run_stream())


RuntimeError: asyncio.run() cannot be called from a running event loop